In [1]:
import mlflow
from mlflow import MlflowClient
import pandas as pd
import numpy as np
from mlflow.models import infer_signature

In [2]:
import optuna
import os


from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [3]:
# get data

file_path = '../data/Loan_Data.csv'
df = pd.read_csv(file_path)

In [4]:
# cleaning a bit

# 1. drop id col
df.drop(columns = ['Loan_ID'], inplace=True)

# 2. drop dupe
df.drop_duplicates(inplace=True)

# 2.1 drop missing values
df.dropna(inplace=True)


# 3. classify num and cat columns

cat_columns = [column for column in df.columns if df[column].dtype == 'object']
num_columns = [column for column in df.columns if df[column].dtype != 'object']

cat_columns.remove('Loan_Status')



# 4. split to training and testing set

X = df.drop(columns = 'Loan_Status').copy()
y = df['Loan_Status'].copy()
        
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                    random_state=42,  
                                    test_size=0.1,
                                    shuffle=True)


# 5. encoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(X_train[cat_columns])

X_train[cat_columns] = encoder.transform(X_train[cat_columns])    
X_test[cat_columns] = encoder.transform(X_test[cat_columns])


# 6. scale

scaler = StandardScaler()
scaler.fit(X_train)

X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)


# Optuna Part + MLFlow

In [5]:
def mlflow_logging(exp_param, model, params:dict, metrics:dict):

    experiment_name = exp_param['name']
    run_name = exp_param['run_name']
    artifact_path = exp_param['artifact_path']
    model_name = exp_param['model_name']
    signature = exp_param['signature']


    # set experiment
    mlflow.set_experiment(experiment_name)



    # initiate the logging
    with mlflow.start_run(run_name = run_name, nested=True) as run:
        
        mlflow.log_params(params)

        mlflow.log_metrics(metrics)


        # Log an instance of the trained model for later use
        mlflow.sklearn.log_model(
            sk_model=model,  
            artifact_path=artifact_path,
            registered_model_name= model_name,
            signature=signature
        )


In [6]:
# set up mlflow client

client = MlflowClient()

In [99]:
# create experiment

experiment_description = (
    "This is the application loan prediction project"
    "This experiment contains the base experiments made"
)

data_clean_made = (
    "remove id col, remove duplicate, [remove missing], categorical encoder, standard scaler"
)

model_involved = (
    "Logistic Regression"
    "AdaBoost"
)

other_tag = (
    "Optuna"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "loan_application_project",
    "team": "syamil",
    "project_quarter": "Q1-2024",
    "mlflow.note.content": experiment_description,
    "data_cleaning" : data_clean_made,
    "mode_involved" : model_involved,
    "other_tag" : other_tag
}

# Create the Experiment, providing a unique name
create_experiment = client.create_experiment(
    name="Loan_Application_Classification", tags=experiment_tags
)


In [7]:
experiment_name = 'first_experiment'
run_name = "first_run"
artifact_path = 'loan_application_model'


In [8]:
def train_model(model_ex, X_train, y_train, X_test, y_test, params:dict):

    #logging.info("Running up Logistic Regression...")

    # run logistic regression model
    model = model_ex.set_params(**params)
    model.fit(X_train, y_train)

    score_train = model.score(X_train, y_train)
    score_valid = model.score(X_test, y_test)


    # trying to get the signature 
    # -> (responsible to save the schema of model input and output)
    predictions = model.predict(X_test)
    signature = infer_signature(X_test, predictions)
    
    
    return model, score_train, score_valid, signature



In [9]:
# Optuna code

def objective(trial, name, model):
    
    #with mlflow.start_run(nested=True):
    
    
        # Adaboost classification
        if name == 'AdaBoost':
            params = {
                'n_estimators' : trial.suggest_int('n_estimators',100, 1000),
                'learning_rate': trial.suggest_float('learning_rate',0.0, 1.0),
                'algorithm' : trial.suggest_categorical('algorithm',['SAMME','SAMME.R'])
            }
            
        # Logistic Regression
        elif name == 'Logistic_Regression':    

            penalty_choices = ['l1', 'l2', 'elasticnet']

            params = {
                #'penalty' : trial.suggest_categorical('penalty', [None, 'l2', 'l1', 'elasticnet']),
                'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
                'solver': trial.suggest_categorical('solver', ['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga']),
                'max_iter': trial.suggest_int('max_iter', 100, 1000)

            }


        model = model.set_params(**params)

        score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

        score
        
        return score


#study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())

#study.optimize(objective, n_trials=5)

In [10]:
run_name = 'base_experiment'

In [11]:
# trying to combine two classifier

names = [
    'Logistic_Regression',
    #'AdaBoost'
]

models = [
    LogisticRegression(),
    #AdaBoostClassifier()
]

optuna_study = {}

def model_training(model, X_train, y_train, X_test, y_test):
    model_fitted = model.fit(X_train, y_train)
    y_pred = model_fitted.predict(X_test)
    score = accuracy_score(y_pred, y_test)
    return model, round(score,3)


for name, model in zip(names, models):
    
    
    with mlflow.start_run(run_name=run_name) as parent_run:
        
        # get mlflow id
        run_id = parent_run.info.run_id
        
        #score = model_training(model,X_train, y_train, X_test, y_test)
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
        objective_fn = lambda trial: objective(trial, name, model)
        study.optimize(objective_fn, n_trials=5)
        
        for i, trial in enumerate(study.trials):
            with mlflow.start_run(run_name=f"{run_name}_{name}_trial_{i+1}",nested=True):
                mlflow.log_params(trial.params)
                mlflow.log_metric('accuracy', trial.value)    
            
                
        
        # update the best model run with unseen data
        
        model, score_train, score_valid, signature = train_model(model, X_train, y_train, X_test, y_test, study.best_params)

        # update the run in experiemnt
        exp_param = {
            'name' : 'Loan_Application_Classification',
            'run_name': f'{run_name}_{name}_best_params',
            'artifact_path': f'loan_application_model/{run_name}',
            'model_name': f'{name}_best_params', 
            'signature': signature
        }
        
        model_params = study.best_params
        
        metrics = {
            'Accuracy Training' : score_train,
            'Accuracy Test' : score_valid 
        }
        
        mlflow_logging(exp_param, model, model_params, metrics)

        optuna_study[name] = study

        #print(f'{name}: {score}')

[I 2024-05-04 14:39:42,166] A new study created in memory with name: no-name-b94a7433-bb39-45fb-a987-392475503109
[I 2024-05-04 14:39:42,283] Trial 0 finished with value: 0.7543972199946538 and parameters: {'class_weight': 'balanced', 'solver': 'sag', 'max_iter': 738}. Best is trial 0 with value: 0.7543972199946538.
[I 2024-05-04 14:39:42,386] Trial 1 finished with value: 0.7543972199946538 and parameters: {'class_weight': 'balanced', 'solver': 'newton-cholesky', 'max_iter': 243}. Best is trial 0 with value: 0.7543972199946538.
[I 2024-05-04 14:39:42,462] Trial 2 finished with value: 0.7986367281475542 and parameters: {'class_weight': None, 'solver': 'newton-cholesky', 'max_iter': 977}. Best is trial 2 with value: 0.7986367281475542.
[I 2024-05-04 14:39:42,555] Trial 3 finished with value: 0.7986367281475542 and parameters: {'class_weight': None, 'solver': 'saga', 'max_iter': 945}. Best is trial 2 with value: 0.7986367281475542.
[I 2024-05-04 14:39:42,624] Trial 4 finished with value: 

In [21]:
optuna_study['Logistic_Regression'].trials_dataframe

<bound method Study.trials_dataframe of <optuna.study.study.Study object at 0x0000025E97BC3A60>>

In [16]:
dir(optuna_study['Logistic_Regression'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ask',
 '_directions',
 '_get_trials',
 '_is_multi_objective',
 '_log_completed_trial',
 '_pop_waiting_trial_id',
 '_should_skip_enqueue',
 '_stop_flag',
 '_storage',
 '_study_id',
 '_tell',
 '_thread_local',
 'add_trial',
 'add_trials',
 'ask',
 'best_params',
 'best_trial',
 'best_trials',
 'best_value',
 'direction',
 'directions',
 'enqueue_trial',
 'get_trials',
 'metric_names',
 'optimize',
 'pruner',
 'sampler',
 'set_metric_names',
 'set_system_attr',
 'set_user_attr',
 'stop',
 'study_name',
 'system_attrs',
 'tell',
 'trials',
 'trials_dataframe',
 'user_attrs']

In [22]:
# test get experiment info

mlflow.search_experiments()

[<Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLOPS%20NEW/E2E%20Loan%20Application%20Classification/E2E_Loan_Application/experiment/mlruns/405433576641488242', creation_time=1714804783920, experiment_id='405433576641488242', last_update_time=1714804783920, lifecycle_stage='active', name='Loan_Application_Classification', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLOPS%20NEW/E2E%20Loan%20Application%20Classification/E2E_Loan_Application/experiment/mlruns/279795776414714051', creation_time=1714740930270, experiment_id='279795776414714051', last_update_time=1714740930270, lifecycle_stage='active', name='EX_2_Loan_Application_Classification_v4', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLOPS%20NEW/E2E%20Loan%20Application%20Classification/E2E_Loan_Application/experiment/mlruns/566338987520218507', creation_time=1714276198462, experiment_id='566338987520218507', last_update_time=1714276198462, lifecycle_stage='a

In [25]:
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    for model_version in model.latest_versions:
        if model_version.current_stage == 'Production':
            deployment_model = model_version
        print(f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}")

name=AdaBoost_best_params; run_id=0a9ead84023342a1b3646b9456a3eef8; version=7, stage=None
name=Logistic_Regression_best_params; run_id=ab1064e8fc734327ae5ca9cf66ba70e7; version=1, stage=Production
name=Logistic_Regression_best_params; run_id=a6076f965e81432793a0d16f93fac0f1; version=10, stage=None


In [26]:
deployment_model

<ModelVersion: aliases=['champion'], creation_timestamp=1713066172078, current_stage='Production', description=None, last_updated_timestamp=1713066612943, name='Logistic_Regression_best_params', run_id='ab1064e8fc734327ae5ca9cf66ba70e7', run_link=None, source='file:///C:/Users/USER/Desktop/MLOPS%20NEW/E2E%20Loan%20Application%20Classification/E2E_Loan_Application/experiment/mlruns/615822914642047264/ab1064e8fc734327ae5ca9cf66ba70e7/artifacts/loan_application_model/base_experiment', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [30]:
model_path = f'runs:/{deployment_model.run_id}/loan_application_model/{run_name}'


loaded_model = mlflow.pyfunc.load_model(model_path)

OSError: No such file or directory: 'C:\Users\USER\Desktop\MLOPS NEW\E2E Loan Application Classification\E2E_Loan_Application\experiment\mlruns\615822914642047264\ab1064e8fc734327ae5ca9cf66ba70e7\artifacts\loan_application_model\base_experiment'

In [33]:
logged_model = 'runs:/a6076f965e81432793a0d16f93fac0f1/loan_application_model/base_experiment'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)





In [34]:
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(loaded_model, open(filename, 'wb'))

In [ ]:
sys